# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [35]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
df = pd.read_csv('weatherfinal.csv')
weather = pd.DataFrame(df)
weather

,city,lon,lat,temp,temp_max,humidity,wind_speed,%_cloud_cover
0,Bluff,168.3333,-46.6000,49.80,49.80,70,19.77,95
1,Bethel,-73.4140,41.3712,43.54,46.72,58,1.01,0
2,Muswellbrook,150.9000,-32.2667,63.25,64.33,91,10.80,66
3,Pangody,74.4859,65.8500,1.92,1.92,97,9.66,97
4,Barra Patuca,-84.2833,15.8000,78.44,78.44,78,11.86,20
...,...,...,...,...,...,...,...,...
505,Port Augusta,137.7667,-32.5000,75.70,75.70,60,13.51,100
506,Lubumbashi,27.4794,-11.6609,63.32,63.32,97,3.29,100
507,Ouégoa,164.4333,-20.3500,80.15,80.15,77,10.85,22
508,Turukhansk,87.9833,65.8167,12.87,12.87,95,13.00,100


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
gmaps.configure(api_key=g_key)

locations = weather[["lat", "lon"]]
humidity = weather["humidity"]


heatmap = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=5)
heatmap.add_layer(heat_layer)


In [10]:
heatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
idealweather = weather.loc[(weather["wind_speed"] <= 10) & (weather["%_cloud_cover"] == 0) & \
                                   (weather["temp_max"] >= 60) & (weather["temp_max"] <= 78)].dropna().reset_index()

idealweather

,index,city,lon,lat,temp,temp_max,humidity,wind_speed,%_cloud_cover
0,11,Cape Town,18.4232,-33.9258,65.62,67.08,69,3.00,0
1,121,Geraldton,114.6000,-28.7667,68.41,68.41,49,6.91,0
2,125,Port Elizabeth,25.5701,-33.9180,68.29,70.11,77,9.22,0
3,162,Los Llanos de Aridane,-17.9182,28.6585,66.13,66.13,39,1.99,0
4,298,Gorom-Gorom,-0.2361,14.4439,68.00,68.00,12,10.00,0
5,371,Coihueco,-71.8333,-36.6167,75.94,75.94,18,6.91,0
6,379,Ténenkou,-4.9169,14.4572,72.66,72.66,10,9.73,0
7,386,Magway,94.9167,20.1500,64.56,64.56,80,2.44,0
8,467,Pacifica,-122.4869,37.6138,56.98,64.56,75,5.01,0
9,493,Sitrah,50.6206,26.1547,62.47,62.60,68,4.00,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [28]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel", "keyword" : "hotel", "radius" : 5000, "key" : g_key}

hotel_df = idealweather
hotel_df["Hotel Name"] = ""

In [43]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["lat"]
    lon = row["lon"]
    city_name = row["city"]
    
    params["location"] = f"{lat},{lon}"
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("moving on")
    time.sleep(1)


moving on
moving on
moving on
moving on
moving on
moving on


In [44]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

hotel_df


,index,city,lon,lat,temp,temp_max,humidity,wind_speed,%_cloud_cover,Hotel Name
0,11,Cape Town,18.4232,-33.9258,65.62,67.08,69,3.00,0,The Bay Hotel
1,121,Geraldton,114.6000,-28.7667,68.41,68.41,49,6.91,0,Ocean Centre Hotel
2,125,Port Elizabeth,25.5701,-33.9180,68.29,70.11,77,9.22,0,The Grand Hotel
3,162,Los Llanos de Aridane,-17.9182,28.6585,66.13,66.13,39,1.99,0,Hotel Hacienda de Abajo
4,298,Gorom-Gorom,-0.2361,14.4439,68.00,68.00,12,10.00,0,
5,371,Coihueco,-71.8333,-36.6167,75.94,75.94,18,6.91,0,
6,379,Ténenkou,-4.9169,14.4572,72.66,72.66,10,9.73,0,
7,386,Magway,94.9167,20.1500,64.56,64.56,80,2.44,0,Phan Khar Myay
8,467,Pacifica,-122.4869,37.6138,56.98,64.56,75,5.01,0,"Pacifica Lighthouse, Trademark Collection by W..."
9,493,Sitrah,50.6206,26.1547,62.47,62.60,68,4.00,0,


In [45]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
heatmap.add_layer(markers)

# Display Map
heatmap




Figure(layout=FigureLayout(height='420px'))